In [1]:
from mongoConnection import MongoConnection
import pandas as pd
db = MongoConnection()

In [10]:
res = db.query_from_to(1667437200,1667444400) #This returns an iterable 'cursor' object 
""" This next bit of code makes a dataframe of all the of stuff from our query!
    Then it strips the dataframe of the stuff we dont need. All you are left with 
    is the time and the high open low close.
    """

ourData = []
for candle in res:
    ourData.append(candle)

df = pd.DataFrame(ourData)

df = pd.concat([df,df["y"].apply(pd.Series)], axis=1)

df = df.drop(columns="y")
print(df)


            x         open         high          low        close
0  1667437200  1518.282446  1535.789490  1515.731501  1531.546037
1  1667440800  1531.295546  1546.301443  1529.771299  1543.499030
2  1667444400  1543.055578  1548.493365  1542.030452  1543.150313
3  1667437200  1518.282446  1535.789490  1515.731501  1531.546037
4  1667440800  1531.295546  1546.301443  1529.771299  1543.499030
5  1667444400  1543.055578  1548.493365  1542.030452  1543.150313
6  1667437200  1518.282446  1535.789490  1515.731501  1531.546037
7  1667440800  1531.295546  1546.301443  1529.771299  1543.499030
8  1667444400  1543.055578  1548.493365  1542.030452  1543.150313


In [9]:
# These are the fast and slow moving averages that we are applying
ma_list = [9,27]

df_ma = df.copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.close.rolling(window=ma).mean()

print(df_ma)


            x         open         high          low        close        MA_9  \
0  1667437200  1518.282446  1535.789490  1515.731501  1531.546037         NaN   
1  1667440800  1531.295546  1546.301443  1529.771299  1543.499030         NaN   
2  1667444400  1543.055578  1548.493365  1542.030452  1543.150313         NaN   
3  1667437200  1518.282446  1535.789490  1515.731501  1531.546037         NaN   
4  1667440800  1531.295546  1546.301443  1529.771299  1543.499030         NaN   
5  1667444400  1543.055578  1548.493365  1542.030452  1543.150313         NaN   
6  1667437200  1518.282446  1535.789490  1515.731501  1531.546037         NaN   
7  1667440800  1531.295546  1546.301443  1529.771299  1543.499030         NaN   
8  1667444400  1543.055578  1548.493365  1542.030452  1543.150313  1539.39846   

   MA_27  
0    NaN  
1    NaN  
2    NaN  
3    NaN  
4    NaN  
5    NaN  
6    NaN  
7    NaN  
8    NaN  
